In [1]:
import pandas as pd
import geopandas as gpd
from census import Census # Or from cenpy import products
import matplotlib.pyplot as plt

# Optional: to make plots appear inline in Jupyter
%matplotlib inline

# Optional: settings to display all columns in pandas
pd.set_option('display.max_columns', None)
print("complete")

complete


In [2]:
# store census api key as an environment variable, you can load it like this:
import os
CENSUS_API_KEY = os.environ.get('CENSUS_API_KEY')
c = Census(CENSUS_API_KEY)
print(CENSUS_API_KEY)
print(c)

41c5cfd9be6ab6ae56b68a3e7eebffc2154bfbe1


In [3]:
# --- Define parameters ---

# Define the range of years for data collection
# ACS 5-Year data, e.g., 2022 data is for 2018-2022, typically released late in the following year.
# Adjust START_YEAR and END_YEAR based on your needs and data availability.
START_YEAR = 2013
END_YEAR = 2022 # As of mid-2024, 2022 ACS 5-year is the latest widely available.

# --- Option 1: Fetch every year in the defined range ---
YEARS_TO_FETCH = list(range(START_YEAR, END_YEAR + 1))

# --- Option 2: Fetch specific years (example) ---
# YEARS_TO_FETCH = [2013, 2017, 2022] # Uncomment and modify this line if you want specific years

# --- Option 3: Fetch every N years (example: every 3 years) ---
# YEARS_TO_FETCH = list(range(START_YEAR, END_YEAR + 1, 3)) # Uncomment and modify for a specific interval

# State FIPS code(s)
# Find FIPS codes here: https://www.nrcs.usda.gov/wps/portal/nrcs/detail/?cid=nrcs143_013696
# Wyoming FIPS code
STATE_FIPS = ['56'] # Wyoming

# County FIPS code(s) for each state
# Laramie County FIPS code (for Cheyenne)
COUNTY_FIPS = {
    '56': ['021'] # Laramie County
    # 'STATE_FIPS_2': ['COUNTY_FIPS_A', 'COUNTY_FIPS_B']
}
# To get all counties in the specified state(s):
# COUNTY_FIPS = {sf: '*' for sf in STATE_FIPS}


# Geographic Level
# Options: 'tract', 'block group', 'county', 'state'
GEO_LEVEL = 'tract' # Census tracts are commonly used for UHI studies

# ACS Variables
# Find variables here:
# - https://api.census.gov/data/YEAR/acs/acs5/variables.html (replace YEAR with your chosen year)
# - Cenpy documentation: https://cenpy-devs.github.io/cenpy/generated/cenpy.products.ACS.html
# Ensure these variables are generally consistent across the years you are fetching.
VARIABLES_CENSUS_API = [
    'NAME', # Name of the geographic area (automatically included by some libraries)
    'B01003_001E', # Total Population
    'B19013_001E', # Median Household Income
    'B17001_002E', # Poverty Count (Income below poverty level)
    'B25002_002E', # Occupied Housing Units
    'B25002_003E', # Vacant Housing Units
    # Add more variables as needed
]

# For `cenpy`, you can often use more descriptive names, check its documentation
# VARIABLES_CENPY = [ # This is not used if you are using the 'census' library primarily
#     'TOTALPOP',
#     'MEDIAN_HOUSEHOLD_INCOME',
#     'POVERTY_TOTAL', # Example, actual name might vary
#     # Add more variables as needed
# ]

print(f"Target Years to Fetch: {YEARS_TO_FETCH}")
print(f"State(s): {STATE_FIPS}")
print(f"Geo Level: {GEO_LEVEL}")
print(f"Variables: {VARIABLES_CENSUS_API}")

Target Years to Fetch: [2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]
State(s): ['56']
Geo Level: tract
Variables: ['NAME', 'B01003_001E', 'B19013_001E', 'B17001_002E', 'B25002_002E', 'B25002_003E']


In [6]:
# CELL 3.5 (Revised - Removing faulty elif check): Iterative Data Acquisition

print("DEBUG: Starting Cell 3.5.")

all_years_data = [] 
acs_df = pd.DataFrame() 

if 'c' not in locals():
    print("DEBUG: Critical Error - Census object 'c' not found. Please run Cell 2 (API Key setup) first.")
else:
    # Assuming 'c' is found, we will now directly try to use other parameters.
    # If any of YEARS_TO_FETCH, STATE_FIPS, etc., are truly missing, 
    # this section will raise a NameError, which is a clear indication.
    print("DEBUG: 'c' object found. Proceeding to use other parameters (YEARS_TO_FETCH, STATE_FIPS, etc.).")
    
    try:
        state_fip_to_fetch = STATE_FIPS[0] 
        county_fip_to_fetch = COUNTY_FIPS[state_fip_to_fetch][0]

        print(f"DEBUG: Preparing to fetch data for State: {state_fip_to_fetch}, County: {county_fip_to_fetch}, Geo Level: {GEO_LEVEL}")
        print(f"DEBUG: Years to fetch: {YEARS_TO_FETCH}")
        print(f"DEBUG: Variables to fetch: {VARIABLES_CENSUS_API}")

        for year_to_process in YEARS_TO_FETCH: 
            print(f"\nFetching data for Year: {year_to_process}...")
            # --- API Call ---
            data_for_year = None # Initialize
            try:
                if GEO_LEVEL == 'tract':
                    data_for_year = c.acs5.state_county_tract(
                        VARIABLES_CENSUS_API,
                        state_fip_to_fetch,
                        county_fip_to_fetch,
                        Census.ALL, 
                        year=year_to_process
                    )
                else:
                    print(f"  Unsupported GEO_LEVEL '{GEO_LEVEL}' for this loop.")
                
                if data_for_year:
                    df_year = pd.DataFrame(data_for_year)
                    df_year['year'] = year_to_process 
                    all_years_data.append(df_year)
                    print(f"  Successfully fetched {len(df_year)} records for {year_to_process}.")
                else:
                    print(f"  No data returned for {year_to_process} for State: {state_fip_to_fetch}, County: {county_fip_to_fetch}.")
            except Exception as e_fetch: # Catch errors during the API call specifically
                print(f"  Error during API call for {year_to_process}: {e_fetch}")
        
        print("\nDEBUG: Finished fetching loop.")

        if all_years_data:
            print("DEBUG: Data found for one or more years. Concatenating now...")
            acs_df = pd.concat(all_years_data, ignore_index=True)
            print(f"\n\nSuccessfully fetched and concatenated data for years: {sorted(acs_df['year'].unique().tolist())}")
            print(f"Total records in combined acs_df: {len(acs_df)}")
            print("--- Info of combined acs_df ---")
            acs_df.info()
            print("\n--- First 5 rows of combined acs_df ---")
            print(acs_df.head())
            print("\n--- Last 5 rows of combined acs_df ---")
            print(acs_df.tail())
        else:
            print("\nDEBUG: No data was successfully fetched across all years. all_years_data list is empty. acs_df will be empty.")
            # Prepare expected columns for an empty DataFrame
            expected_cols = ['NAME'] + [v for v in VARIABLES_CENSUS_API if v != 'NAME'] + ['state', 'county', 'tract', 'year']
            seen = set()
            expected_cols_final = [x for x in expected_cols if not (x in seen or seen.add(x))]
            acs_df = pd.DataFrame(columns=expected_cols_final)
            print(f"DEBUG: Initialized acs_df as empty DataFrame with columns: {list(acs_df.columns)}")

    except NameError as ne:
        print(f"DEBUG: NameError encountered - A required parameter (like YEARS_TO_FETCH, STATE_FIPS, etc.) is truly missing or not defined: {ne}")
        print("DEBUG: Please ensure your parameters cell was run correctly before this cell.")
    except Exception as e_outer:
        print(f"DEBUG: An unexpected error occurred in the main data processing block: {e_outer}")


print("DEBUG: Reached end of Cell 3.5.")

DEBUG: Starting Cell 3.5.
DEBUG: 'c' object found. Proceeding to use other parameters (YEARS_TO_FETCH, STATE_FIPS, etc.).
DEBUG: Preparing to fetch data for State: 56, County: 021, Geo Level: tract
DEBUG: Years to fetch: [2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]
DEBUG: Variables to fetch: ['NAME', 'B01003_001E', 'B19013_001E', 'B17001_002E', 'B25002_002E', 'B25002_003E']

Fetching data for Year: 2013...
  Successfully fetched 21 records for 2013.

Fetching data for Year: 2014...
  Successfully fetched 21 records for 2014.

Fetching data for Year: 2015...
  Successfully fetched 21 records for 2015.

Fetching data for Year: 2016...
  Successfully fetched 21 records for 2016.

Fetching data for Year: 2017...
  Successfully fetched 21 records for 2017.

Fetching data for Year: 2018...
  Successfully fetched 21 records for 2018.

Fetching data for Year: 2019...
  Successfully fetched 21 records for 2019.

Fetching data for Year: 2020...
  Successfully fetched 27 records f

In [7]:
# Ensure acs_df exists from one of the methods above
if 'acs_df' not in locals() or acs_df.empty:
    print("acs_df is not defined or is empty. Please run a data acquisition step (3.A or 3.B).Else, if using cenpy for geometries, ensure acs_gdf is populated.")
else:
    print("Starting data cleaning and preparation...")
    # Display basic info
    print("\n--- Initial DataFrame Info ---")
    acs_df.info()
    print("\n--- First 5 Rows ---")
    print(acs_df.head())
    print("\n--- Descriptive Statistics (before type conversion) ---")
    print(acs_df.describe(include='all'))

    # Rename columns for clarity (adjust based on VARIABLES_CENSUS_API)
    # Example:
    column_renames = {
        'B01003_001E': 'Total_Population',
        'B19013_001E': 'Median_Household_Income',
        'B17001_002E': 'Poverty_Count',
        'B25002_002E': 'Occupied_Housing_Units',
        'B25002_003E': 'Vacant_Housing_Units',
        # Add other renames as needed
    }
    acs_df.rename(columns=column_renames, inplace=True)
    print("\n--- Columns after renaming ---")
    print(acs_df.columns)

    # Identify numeric columns (all except 'NAME', 'state', 'county', 'tract', etc.)
    # This depends on the variables you fetched.
    # Be careful: 'GEOID' or similar identifiers should remain strings for merging.
    numeric_cols = [col for col in column_renames.values() if col in acs_df.columns]

    # Convert numeric columns to appropriate type, handling errors
    for col in numeric_cols:
        acs_df[col] = pd.to_numeric(acs_df[col], errors='coerce') # 'coerce' will turn errors into NaT/NaN

    print("\n--- DataFrame Info after type conversion ---")
    acs_df.info()
    print("\n--- Descriptive Statistics (after type conversion) ---")
    print(acs_df[numeric_cols].describe())

    # Handle Missing Values
    # Census data sometimes uses large negative numbers for missing/suppressed data.
    # pd.to_numeric with errors='coerce' handles some of this by making them NaN.
    print("\n--- Missing Values per Column ---")
    print(acs_df.isnull().sum())

    # Example: If you know specific placeholder values for missing data (e.g., -999999999)
    # for col in numeric_cols:
    #     acs_df[col] = acs_df[col].replace([-999999999, -888888888, -666666666], pd.NA)


    # Create GEOID for merging (if not already present and consistent)
    # The census library usually provides 'state', 'county', and 'tract' (or 'block group') columns.
    # A common GEOID format is state (2 digits) + county (3 digits) + tract (6 digits).
    if GEO_LEVEL == 'tract' and all(c in acs_df.columns for c in ['state', 'county', 'tract']):
        acs_df['GEOID'] = acs_df['state'] + acs_df['county'] + acs_df['tract']
        print("\nCreated GEOID column for tracts.")
    elif GEO_LEVEL == 'block group' and all(c in acs_df.columns for c in ['state', 'county', 'tract', 'block group']):
        acs_df['GEOID'] = acs_df['state'] + acs_df['county'] + acs_df['tract'] + acs_df['block group']
        print("\nCreated GEOID column for block groups.")
    elif 'GEO_ID' in acs_df.columns and 'GEOID' not in acs_df.columns: # some API versions might use GEO_ID
        # The format of GEO_ID can be "1400000US<state_fip><county_fip><tract_id>"
        # We need to extract the numeric part.
        acs_df['GEOID'] = acs_df['GEO_ID'].apply(lambda x: x.split('US')[-1] if isinstance(x, str) and 'US' in x else x)
        print("\nExtracted GEOID from GEO_ID column.")
    elif 'GEOID' in acs_df.columns:
        print("\nGEOID column already exists.")
    else:
        print("\nWarning: Could not create GEOID. Ensure 'state', 'county', and 'tract'/'block group' columns exist or GEOID is provided.")

    if 'GEOID' in acs_df.columns:
        print(acs_df[['GEOID'] + numeric_cols].head())

    # Drop rows where essential numeric data or GEOID is missing if necessary
    # For example, if Total_Population is NaN, the record might not be useful.
    # acs_df.dropna(subset=['GEOID', 'Total_Population'], inplace=True)
    # print(f"\nShape of ACS data after dropping critical NaNs: {acs_df.shape}")

Starting data cleaning and preparation...

--- Initial DataFrame Info ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 228 entries, 0 to 227
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   NAME         228 non-null    object 
 1   B01003_001E  228 non-null    float64
 2   B19013_001E  228 non-null    float64
 3   B17001_002E  228 non-null    float64
 4   B25002_002E  228 non-null    float64
 5   B25002_003E  228 non-null    float64
 6   state        228 non-null    object 
 7   county       228 non-null    object 
 8   tract        228 non-null    object 
 9   year         228 non-null    int64  
dtypes: float64(5), int64(1), object(4)
memory usage: 17.9+ KB

--- First 5 Rows ---
                                       NAME  B01003_001E  B19013_001E  \
0  Census Tract 20, Laramie County, Wyoming       7442.0      61739.0   
1   Census Tract 6, Laramie County, Wyoming       6177.0      43650.0   
2   Census Tr

In [8]:
# 5. Handle Specific Placeholder Negative Values for Missing Data

if 'acs_df' not in locals() or acs_df.empty:
    print("acs_df is not defined or is empty. Please run the preceding cells first.")
else:
    print("Looking for and replacing known Census placeholder negative values...")
    
    # Define known placeholder negative values used by ACS for missing/suppressed data
    # These are common ones, but you might encounter others.
    # -666666666: Data not available or suppressed
    # -888888888: Not applicable
    # -999999999: Margin of error could not be calculated / data suppressed
    census_null_placeholders = [-999999999.0, -888888888.0, -666666666.0]
    
    # Identify which columns to apply this to (your current numeric_cols)
    # numeric_cols was defined in the previous cell
    if 'numeric_cols' not in locals():
        print("Warning: 'numeric_cols' not found. Re-identifying numeric columns based on current dtypes.")
        # Fallback: re-identify numeric columns if 'numeric_cols' isn't in scope
        # (This can happen if cells are run non-sequentially after a kernel restart)
        column_renames_ref = { # Re-define for safety if not in scope
            'B01003_001E': 'Total_Population',
            'B19013_001E': 'Median_Household_Income',
            'B17001_002E': 'Poverty_Count',
            'B25002_002E': 'Occupied_Housing_Units',
            'B25002_003E': 'Vacant_Housing_Units',
        }
        numeric_cols = [col for col in column_renames_ref.values() if col in acs_df.columns and pd.api.types.is_numeric_dtype(acs_df[col])]

    cols_to_check = [col for col in numeric_cols if col in acs_df.columns]
    
    if not cols_to_check:
        print("No numeric columns identified to check for placeholders.")
    else:
        print(f"Checking columns: {cols_to_check}")
        for col in cols_to_check:
            # Count how many placeholders are found before replacing
            initial_placeholders = acs_df[col].isin(census_null_placeholders).sum()
            if initial_placeholders > 0:
                print(f"  Found {initial_placeholders} placeholder(s) in '{col}'. Replacing with NaN.")
                acs_df[col] = acs_df[col].replace(census_null_placeholders, pd.NA) # or np.nan
            else:
                print(f"  No known placeholders found in '{col}'.")

    print("\n--- Missing Values per Column (after replacing placeholders) ---")
    print(acs_df.isnull().sum())

    print("\n--- Descriptive Statistics (after replacing placeholders) ---")
    # Ensure numeric_cols only contains columns that still exist and are numeric
    final_numeric_cols_for_describe = [col for col in numeric_cols if col in acs_df.columns and pd.api.types.is_numeric_dtype(acs_df[col])]
    if final_numeric_cols_for_describe:
        print(acs_df[final_numeric_cols_for_describe].describe())
    else:
        print("No numeric columns available for description.")
        
    print("\n--- First 5 Rows (after replacing placeholders) ---")
    print(acs_df.head())

Looking for and replacing known Census placeholder negative values...
Checking columns: ['Total_Population', 'Median_Household_Income', 'Poverty_Count', 'Occupied_Housing_Units', 'Vacant_Housing_Units']
  No known placeholders found in 'Total_Population'.
  Found 11 placeholder(s) in 'Median_Household_Income'. Replacing with NaN.
  No known placeholders found in 'Poverty_Count'.
  No known placeholders found in 'Occupied_Housing_Units'.
  No known placeholders found in 'Vacant_Housing_Units'.

--- Missing Values per Column (after replacing placeholders) ---
NAME                        0
Total_Population            0
Median_Household_Income    11
Poverty_Count               0
Occupied_Housing_Units      0
Vacant_Housing_Units        0
state                       0
county                      0
tract                       0
year                        0
GEOID                       0
dtype: int64

--- Descriptive Statistics (after replacing placeholders) ---
       Total_Population  Pover

In [9]:
# 6. Calculate Derived Variables

if 'acs_df' not in locals() or acs_df.empty:
    print("acs_df is not defined or is empty. Please run the preceding cells first.")
elif not all(col in acs_df.columns for col in ['Total_Population', 'Poverty_Count', 'Occupied_Housing_Units', 'Vacant_Housing_Units']):
    print("One or more required columns for calculating derived variables are missing. Ensure renaming and cleaning steps were successful.")
    print(f"Required: 'Total_Population', 'Poverty_Count', 'Occupied_Housing_Units', 'Vacant_Housing_Units'. Found: {list(acs_df.columns)}")
else:
    print("Calculating derived variables...")

    # Ensure Total_Population is numeric and not zero for poverty rate calculation
    if 'Total_Population' in acs_df.columns:
        if pd.api.types.is_numeric_dtype(acs_df['Total_Population']):
            # Calculate Poverty Rate
            # Handle division by zero: if Total_Population is 0 or NaN, result should be NaN
            acs_df['Poverty_Rate'] = acs_df['Poverty_Count'] / acs_df['Total_Population']
            # Replace potential infinity values (if Poverty_Count > 0 and Total_Population == 0) with NaN
            acs_df['Poverty_Rate'].replace([float('inf'), float('-inf')], pd.NA, inplace=True) 
            # Convert to percentage if desired (e.g., multiply by 100)
            # acs_df['Poverty_Rate_Percent'] = acs_df['Poverty_Rate'] * 100
            print("  Calculated 'Poverty_Rate'.")
        else:
            print("  'Total_Population' is not numeric. Skipping Poverty Rate calculation.")
    else:
        print("  'Total_Population' column missing. Skipping Poverty Rate calculation.")

    # Calculate Housing Vacancy Rate
    if all(col in acs_df.columns for col in ['Occupied_Housing_Units', 'Vacant_Housing_Units']):
        if all(pd.api.types.is_numeric_dtype(acs_df[col]) for col in ['Occupied_Housing_Units', 'Vacant_Housing_Units']):
            acs_df['Total_Housing_Units_Calc'] = acs_df['Occupied_Housing_Units'] + acs_df['Vacant_Housing_Units']
            # Handle division by zero: if Total_Housing_Units_Calc is 0 or NaN, result should be NaN
            acs_df['Vacancy_Rate'] = acs_df['Vacant_Housing_Units'] / acs_df['Total_Housing_Units_Calc']
            acs_df['Vacancy_Rate'].replace([float('inf'), float('-inf')], pd.NA, inplace=True)
            # Convert to percentage if desired
            # acs_df['Vacancy_Rate_Percent'] = acs_df['Vacancy_Rate'] * 100
            print("  Calculated 'Vacancy_Rate' (and helper 'Total_Housing_Units_Calc').")
        else:
            print("  'Occupied_Housing_Units' or 'Vacant_Housing_Units' is not numeric. Skipping Vacancy Rate.")
    else:
        print("  'Occupied_Housing_Units' or 'Vacant_Housing_Units' column missing. Skipping Vacancy Rate calculation.")
        
    # Add more derived variables as needed
    # For example, percentage of population with a certain characteristic, etc.

    print("\n--- First 5 Rows with Derived Variables ---")
    print(acs_df.head())

    print("\n--- Descriptive Statistics of Key Numeric Columns (including new rates) ---")
    cols_to_describe_final = ['Total_Population', 'Median_Household_Income', 'Poverty_Count', 
                              'Occupied_Housing_Units', 'Vacant_Housing_Units', 
                              'Poverty_Rate', 'Vacancy_Rate']
    # Filter out any columns that might not exist (e.g., if a rate calculation was skipped)
    # or are not numeric anymore (shouldn't happen with pd.NA in float columns)
    final_cols_exist_and_numeric = [
        col for col in cols_to_describe_final 
        if col in acs_df.columns and pd.api.types.is_numeric_dtype(acs_df[col])
    ]
    if final_cols_exist_and_numeric:
        print(acs_df[final_cols_exist_and_numeric].describe())
    else:
        print("No numeric columns available for description.")
        
    print("\n--- Missing values after deriving variables ---")
    print(acs_df.isnull().sum())


Calculating derived variables...
  Calculated 'Poverty_Rate'.
  Calculated 'Vacancy_Rate' (and helper 'Total_Housing_Units_Calc').

--- First 5 Rows with Derived Variables ---
                                       NAME  Total_Population  \
0  Census Tract 20, Laramie County, Wyoming            7442.0   
1   Census Tract 6, Laramie County, Wyoming            6177.0   
2   Census Tract 7, Laramie County, Wyoming            4053.0   
3   Census Tract 8, Laramie County, Wyoming            1706.0   
4  Census Tract 10, Laramie County, Wyoming            3374.0   

  Median_Household_Income  Poverty_Count  Occupied_Housing_Units  \
0                 61739.0          321.0                  2777.0   
1                 43650.0          696.0                  2690.0   
2                 33088.0          532.0                  1915.0   
3                 62015.0          120.0                   811.0   
4                 40848.0          522.0                  1443.0   

   Vacant_Housing_Units 

C:\Users\TR\AppData\Local\Temp\ipykernel_15748\3636432842.py:18: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  acs_df['Poverty_Rate'].replace([float('inf'), float('-inf')], pd.NA, inplace=True)
C:\Users\TR\AppData\Local\Temp\ipykernel_15748\3636432842.py:33: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values alwa

In [12]:
# 7. Select and Reorder Columns for Final Output

if 'acs_df' not in locals() or acs_df.empty:
    print("acs_df is not defined or is empty. Please run the preceding cells first.")
else:
    print("Selecting and reordering columns for the final dataset...")
    
    # Define the desired order of columns
    # Include 'GEOID' as the primary key, followed by key indicators and rates.
    # Adjust this list based on your analytical needs.
    final_columns = [
        'GEOID',
        'NAME',
        'Total_Population',
        'Median_Household_Income',
        'Poverty_Count',
        'Poverty_Rate',
        'Total_Housing_Units_Calc', # Or 'B25003_001E' if you fetched it as 'Total Housing Units'
        'Occupied_Housing_Units',
        'Vacant_Housing_Units',
        'Vacancy_Rate',
        'tract',
        'year'
        # Add any other original or derived columns you want to keep
        # 'state', 'county', 'tract' # (optional, as GEOID contains this info)
    ]
    
    # Filter out any columns in final_columns that might not exist in acs_df
    # (e.g., if a variable wasn't fetched or a derived variable calculation was skipped)
    existing_final_columns = [col for col in final_columns if col in acs_df.columns]
    
    acs_final_df = acs_df[existing_final_columns].copy() # Use .copy() to avoid SettingWithCopyWarning
    
    print("\n--- First 5 Rows of the Final Selected DataFrame ---")
    print(acs_final_df.head())
    
    print("\n--- Info of the Final Selected DataFrame ---")
    acs_final_df.info()
    
    print("\n--- Missing Values in Final Selected DataFrame ---")
    print(acs_final_df.isnull().sum())


Selecting and reordering columns for the final dataset...

--- First 5 Rows of the Final Selected DataFrame ---
         GEOID                                      NAME  Total_Population  \
0  56021002000  Census Tract 20, Laramie County, Wyoming            7442.0   
1  56021000600   Census Tract 6, Laramie County, Wyoming            6177.0   
2  56021000700   Census Tract 7, Laramie County, Wyoming            4053.0   
3  56021000800   Census Tract 8, Laramie County, Wyoming            1706.0   
4  56021001000  Census Tract 10, Laramie County, Wyoming            3374.0   

  Median_Household_Income  Poverty_Count  Poverty_Rate  \
0                 61739.0          321.0      0.043134   
1                 43650.0          696.0      0.112676   
2                 33088.0          532.0      0.131261   
3                 62015.0          120.0      0.070340   
4                 40848.0          522.0      0.154713   

   Total_Housing_Units_Calc  Occupied_Housing_Units  Vacant_Housing_Un

In [13]:
# 8. Save the Cleaned Data

if 'acs_final_df' not in locals() or acs_final_df.empty:
    print("acs_final_df is not defined or is empty. Please run the column selection cell (7) first.")
else:
    # Define output path and filename
    # Consider creating a 'processed_data' subdirectory if you haven't already
    output_directory = 'final' # Matches your project structure
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)
        print(f"Created directory: {output_directory}")

    output_filename = os.path.join(output_directory, f"wy_laramie_acs_ALLYEARS_tract_data_cleaned.csv")
    
    try:
        acs_final_df.to_csv(output_filename, index=False)
        print(f"\nSuccessfully saved cleaned data to: {output_filename}")
    except Exception as e:
        print(f"\nError saving data to CSV: {e}")

    # Optional: Save to other formats like Excel
    # output_excel_filename = os.path.join(output_directory, f"wy_laramie_acs{YEAR}_tract_data_cleaned.xlsx")
    # try:
    #     acs_final_df.to_excel(output_excel_filename, index=False)
    #     print(f"Successfully saved cleaned data to: {output_excel_filename}")
    # except Exception as e:
    #     print(f"Error saving data to Excel: {e}")



Successfully saved cleaned data to: final\wy_laramie_acs_ALLYEARS_tract_data_cleaned.csv
